In [50]:
import pandas as pd
from sqlalchemy import create_engine
from census import Census
from config_file import (census_key)
import us
import requests

In [51]:
#load file
file_path = 'usa_county_wise.csv'
covid_data = pd.read_csv(file_path)
covid_data

,UID,iso2,iso3,code3,FIPS,Admin2,Province_State,Country_Region,Lat,Long_,Combined_Key,Date,Confirmed,Deaths
0,16,AS,ASM,16,60.0,NaN,American Samoa,US,-14.271000,-170.132000,"American Samoa, US",1/22/20,0,0
1,316,GU,GUM,316,66.0,NaN,Guam,US,13.444300,144.793700,"Guam, US",1/22/20,0,0
2,580,MP,MNP,580,69.0,NaN,Northern Mariana Islands,US,15.097900,145.673900,"Northern Mariana Islands, US",1/22/20,0,0
3,63072001,PR,PRI,630,72001.0,Adjuntas,Puerto Rico,US,18.180117,-66.754367,"Adjuntas, Puerto Rico, US",1/22/20,0,0
4,63072003,PR,PRI,630,72003.0,Aguada,Puerto Rico,US,18.360255,-67.175131,"Aguada, Puerto Rico, US",1/22/20,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
627915,84070016,US,USA,840,NaN,Central Utah,Utah,US,39.372319,-111.575868,"Central Utah, Utah, US",7/27/20,347,1
627916,84070017,US,USA,840,NaN,Southeast Utah,Utah,US,38.996171,-110.701396,"Southeast Utah, Utah, US",7/27/20,70,0
627917,84070018,US,USA,840,NaN,Southwest Utah,Utah,US,37.854472,-111.441876,"Southwest Utah, Utah, US",7/27/20,2781,23
627918,84070019,US,USA,840,NaN,TriCounty,Utah,US,40.124915,-109.517442,"TriCounty, Utah, US",7/27/20,142,0


In [52]:
#clean data
filter = (covid_data.iso2 == "US")
c1 = covid_data[filter]
c2 = c1[['FIPS', 'Admin2', 'Province_State', 'Lat', 'Long_', 'Date', 'Confirmed', 'Deaths']]
c3 = c2.rename(columns ={
    "Admin2":"city",
    "Province_State":"state",
    "Long_":"lng",
    "Lat":'lat'
})
filter =  (c3.Date == "7/27/20")
c4 = c3[filter]
c5 = c4.dropna()
c6 = c5.reset_index()
c6.FIPS = c6.FIPS.astype(int)


covid_clean = c6

In [53]:
# Census API Key
c = Census(census_key, year=2018)

# US states abbrv mapping
states = us.states.mapping('abbr', 'name')
states = {state: abbrev for abbrev, state in states.items()}

In [54]:
# Census search for poverty data based on state, county, fips data
census_data_fips = c.acs5.get(("NAME", "B01003_001E", "B17001_002E"), {'for': 'state:*', 'for':'county:*'})

# Convert to DataFrame
census_fips_pd = pd.DataFrame(census_data_fips)

# Column Reordering
census_fips_pd = census_fips_pd.rename(columns={"B01003_001E": "Population",
                                      "B17001_002E": "Poverty Count",
                                      "NAME": "Name",
                                      "state": "State ID",
                                      "county": "County ID"})

# Drop any rows with NAN
census_fips_pd.dropna(how='any', inplace=True)

# Add in Poverty Rate (Poverty Count / Population)
census_fips_pd["Poverty Rate"] = 100 * (census_fips_pd["Poverty Count"].astype(
        int) / census_fips_pd["Population"].astype(int))

# Final DataFrame
census_fips_pd = census_fips_pd[["Name", "State ID", "County ID", "Population", "Poverty Rate"]]

census_fips_pd

,Name,State ID,County ID,Population,Poverty Rate
0,"Washington County, Mississippi",28,151,47086.0,32.909994
1,"Perry County, Mississippi",28,111,12028.0,18.548387
2,"Choctaw County, Mississippi",28,019,8321.0,22.689581
3,"Itawamba County, Mississippi",28,057,23480.0,14.599659
4,"Carroll County, Mississippi",28,015,10129.0,14.581894
...,...,...,...,...,...
3215,"Clayton County, Iowa",19,043,17672.0,8.850158
3216,"Buena Vista County, Iowa",19,021,20260.0,12.374136
3217,"Guthrie County, Iowa",19,077,10674.0,11.008057
3218,"Humboldt County, Iowa",19,091,9566.0,11.530420


In [55]:
# Data Cleanup
def get_county(address):
    county = address.split("County")[0].strip(" ")
    county = strip_state(county)
    return county

def strip_state(address):
    return address.split(",")[0].strip(" ")

def get_state(address):
    state = address.split(",")[1]
    # Special case for District of Columbia
    state = state[1:]
    return state

census_fips_pd['County'] = census_fips_pd['Name'].apply(lambda x: f"{get_county(x)}")
census_fips_pd['State Name'] = census_fips_pd['Name'].apply(lambda x: f"{get_state(x)}")

#Drop PR
indicesPR = census_fips_pd.index[census_fips_pd['State ID'] == '72' ].tolist()
census_fips_pd.drop(indicesPR, inplace=True)

# Get state abbreviations
census_fips_pd['State'] = census_fips_pd['State Name'].map(states)

census_clean_pd = census_fips_pd.drop(['Name', 'State Name'], inplace=False, axis = 1)

census_clean_pd = census_clean_pd[["State", "State ID", "County", "County ID", "Population", "Poverty Rate"]]

census_clean_pd['FIPS'] = census_clean_pd['State ID'] + census_clean_pd['County ID']

census_clean_pd = census_clean_pd[["State", "County", "FIPS", "Population", "Poverty Rate"]]

census_clean_pd

,State,County,FIPS,Population,Poverty Rate
0,MS,Washington,28151,47086.0,32.909994
1,MS,Perry,28111,12028.0,18.548387
2,MS,Choctaw,28019,8321.0,22.689581
3,MS,Itawamba,28057,23480.0,14.599659
4,MS,Carroll,28015,10129.0,14.581894
...,...,...,...,...,...
3215,IA,Clayton,19043,17672.0,8.850158
3216,IA,Buena Vista,19021,20260.0,12.374136
3217,IA,Guthrie,19077,10674.0,11.008057
3218,IA,Humboldt,19091,9566.0,11.530420


In [56]:
census_clean_pd.FIPS = census_clean_pd.FIPS.astype(int)

merged_df = pd.merge(census_clean_pd, covid_clean, how='inner', on=['FIPS','FIPS'])

In [57]:
#clean merged df
merged_df = merged_df[['FIPS', 'Confirmed', 'Deaths', 'Poverty Rate', 'State', 'County']]
merged_df.rename(columns={'FIPS': 'fips', 'Confirmed': 'confirmed', 'Deaths': 'deaths', 'Poverty Rate': 'poverty_rate', 'State': 'state', 'County': 'county'}, inplace=True)
merged_df

,fips,confirmed,deaths,poverty_rate,state,county
0,28151,1283,24,32.909994,MS,Washington
1,28111,179,7,18.548387,MS,Perry
2,28019,106,4,22.689581,MS,Choctaw
3,28057,257,9,14.599659,MS,Itawamba
4,28015,232,11,14.581894,MS,Carroll
...,...,...,...,...,...,...
3136,19043,82,3,8.850158,IA,Clayton
3137,19021,1776,12,12.374136,IA,Buena Vista
3138,19077,122,5,11.008057,IA,Guthrie
3139,19091,82,1,11.530420,IA,Humboldt


In [58]:
#get data into SQL
rds_connection_string = "postgres:Ru5tC@hle@localhost:5432/covid"
engine = create_engine(f'postgresql://{rds_connection_string}')
engine.table_names()
merged_df.to_sql(name='covid_census', con=engine, if_exists='append', index=False)
pd.read_sql_query('select * from covid_census', con=engine).head()

IntegrityError: (psycopg2.errors.UniqueViolation) duplicate key value violates unique constraint "covid_census_pkey"
DETAIL:  Key (fips)=(28151) already exists.

[SQL: INSERT INTO covid_census (fips, confirmed, deaths, poverty_rate, state, county) VALUES (%(fips)s, %(confirmed)s, %(deaths)s, %(poverty_rate)s, %(state)s, %(county)s)]
[parameters: ({'fips': 28151, 'confirmed': 1283, 'deaths': 24, 'poverty_rate': 32.90999447818885, 'state': 'MS', 'county': 'Washington'}, {'fips': 28111, 'confirmed': 179, 'deaths': 7, 'poverty_rate': 18.548387096774192, 'state': 'MS', 'county': 'Perry'}, {'fips': 28019, 'confirmed': 106, 'deaths': 4, 'poverty_rate': 22.689580579257303, 'state': 'MS', 'county': 'Choctaw'}, {'fips': 28057, 'confirmed': 257, 'deaths': 9, 'poverty_rate': 14.599659284497445, 'state': 'MS', 'county': 'Itawamba'}, {'fips': 28015, 'confirmed': 232, 'deaths': 11, 'poverty_rate': 14.581893572909468, 'state': 'MS', 'county': 'Carroll'}, {'fips': 28043, 'confirmed': 770, 'deaths': 17,